## 1. install tensorflow 

In [ ]:
!pip install tensorflow



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## 2. keras classifier pipeline

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
import numpy as np


def classify(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    img_preprocessed = resnet50.preprocess_input(img_batch)

    # Load ResNet50 model (pretrained on ImageNet)
    model = resnet50.ResNet50()

    # Make predictions
    prediction = model.predict(img_preprocessed)

    # Decode and print top-3 predictions
    print(resnet50.decode_predictions(prediction, top=3)[0])

# usage
classify("./image_data.png")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 814ms/step
[('n02123045', 'tabby', np.float32(0.4986208)), ('n02124075', 'Egyptian_cat', np.float32(0.17687717)), ('n02123159', 'tiger_cat', np.float32(0.06452155))]
